# Import Modules

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import json
import glob

# Read Data

In [2]:
data = {}
for path in glob.glob(r"../../Data/t_taxi/*.csv"):
    data[path.split('\\')[-1].split('.')[0]] = pd.read_csv(path)
data.keys()

dict_keys(['processed_dep_h0', 'processed_dep_h120', 'processed_dep_h180', 'processed_dep_h30', 'raw_dep_h0', 'raw_dep_h120', 'raw_dep_h180', 'raw_dep_h30'])

In [3]:
with open("../../Data/t_taxi/csv_docs.json", "r") as f:
    docs = json.load(f)

# Model Evaluation Function

In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time

def model_eval(y, y_pred, name=None, file=None, verbose=True, **kwargs):
    report = {}
    if name:
        report['name'] = name
        if verbose:
            print(name)
    
    report["RMSE"] = mean_squared_error(y, y_pred, squared=False)
    report["MAE"] = mean_absolute_error(y, y_pred)
    report["% <2 min"] = sum(abs(y-y_pred) < 2*60)/len(y)*100
    report["% <5 min"] = sum(abs(y-y_pred) < 5*60)/len(y)*100
    report["% <7 min"] = sum(abs(y-y_pred) < 7*60)/len(y)*100
    report["time"] = str(pd.Timestamp(round(time.time()), unit='s'))
    
    for kwarg in kwargs:
        report[kwarg] = kwargs[kwarg]
    
    if file is not None:
        with open(file, "a") as f:
            f.write(str(report)+"\n")
    if verbose:
        print(report)
    return(report)

# Preprocessing Function

In [5]:
def gen_preprocessor(X_columns):
    ColumnTransformations = []
    num_cols = []
    cat_cols = []
    
    from sklearn.preprocessing import QuantileTransformer
    for col in X_columns:
        if 'circular' in docs[col]['type']:
            ColumnTransformations.append(
            (
                col + '_qcut_' + str(docs[col]['n_bins']),
                QuantileTransformer(n_quantiles=docs[col]['n_bins']),
                [col]
            ))
        if 'num' in docs[col]['type']:
            num_cols.append(col)
        if 'cat' in docs[col]['type']:
            cat_cols.append(col)

    from sklearn.preprocessing import StandardScaler
    from sklearn.impute import SimpleImputer
    from sklearn.pipeline import Pipeline
    num_trans = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])
    ColumnTransformations.append(("num_trans", num_trans, num_cols))

    from sklearn.preprocessing import OneHotEncoder
    ColumnTransformations.append(("onehotencode", OneHotEncoder(handle_unknown='ignore'), cat_cols))

    from sklearn.compose import ColumnTransformer
    return ColumnTransformer(ColumnTransformations, remainder='drop', n_jobs=-1)

# Data Selection and Slicing

In [6]:
h = 30
df = data['raw_dep_h{}'.format(h)].copy()
df['center_crossing'][(df['center_crossing']=='-')|(df['trwy']!='36L')] = np.nan
df['trwy_ext'] = df['trwy'] + ("_" + df['center_crossing']).fillna('')
df.sort_values('t_predict', inplace=True)

train = pd.DataFrame(index=df.iloc[:int(df.shape[0]*.7)].index)
train['dtype'] = "TRAIN"
val = pd.DataFrame(index=df.iloc[int(df.shape[0]*.7):int(df.shape[0]*0.85)].index)
val['dtype'] = "VALIDATE"
test = pd.DataFrame(index=df.iloc[int(df.shape[0]*0.85):].index)
test['dtype'] = "TEST"
df = pd.concat([df, pd.concat([train, val, test])], axis=1)

<ipython-input-6-e4d2a3ab9c9d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['center_crossing'][(df['center_crossing']=='-')|(df['trwy']!='36L')] = np.nan


In [7]:
X_cols_circ = []
X_cols_num = []
X_cols_cat = []
astype_dict = {}
for col in df.columns:
    if "dtype" == col:
        continue
    if 'circular' in docs[col]['type']:
        X_cols_circ.append(col)
        X_cols_cat.append(col)
        astype_dict[col] = 'category'
    elif "num" in docs[col]['type']:
        X_cols_num.append(col)
        astype_dict[col] = np.float64
    elif "cat" in docs[col]['type']:
        X_cols_cat.append(col)
        astype_dict[col] = "category"
    

X_cols = X_cols_cat + X_cols_num

In [8]:
X_train = df[df['dtype']=='TRAIN']
y_train = X_train.pop('t_taxi')

X_val = df[df['dtype']=='VALIDATE']
y_val = X_val.pop('t_taxi')

X_test = df[df['dtype']=='TEST']
y_test = X_test.pop('t_taxi')

# Linear Model, No Performance Feedback

In [9]:
from sklearn.linear_model import HuberRegressor
from sklearn.pipeline import Pipeline

alpha = .4
epsilon = 1.

regr = HuberRegressor(
    max_iter=10000,
    alpha=alpha,
    epsilon=epsilon,
)

X_cols_sel = [
    'actype',
    'depgnr',
    'n_dep',
    'dew',
    'lightning',
    'rvr5000_2000',
    'plr1',
    'plr2',
    'local_mod',
    'trwy_ext',
]

t0 = time.time()
model = Pipeline(steps=[("preprocessor", gen_preprocessor(X_cols_sel)),
                       ("regressor", regr)])

model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)

no_fs_score = model_eval(
    y_val, 
    y_val_pred,
    name='no_pf_train',
)['% <2 min']

no_pf_train
{'name': 'no_pf_train', 'RMSE': 183.6191804345149, 'MAE': 124.38923499757205, '% <2 min': 61.56967262023151, '% <5 min': 92.66478380206054, '% <7 min': 97.1677473248737, 'time': '2021-04-29 10:43:44'}


# Performance Feedback

## Performance Feedback PostgreSQL Table

In [10]:
import psycopg2
from sqlalchemy import create_engine

X_val_pf = pd.concat([X_train, X_val])[['t_taxi_end', 't_predict', 'dtype']]
X_val_pf['t_taxi_end'] = pd.to_datetime(X_val_pf['t_taxi_end'])
X_val_pf['t_predict'] = pd.to_datetime(X_val_pf['t_predict'])
X_val_pf['error'] = pd.concat([y_train, y_val]) - np.hstack([y_train_pred, y_val_pred])

table = 'lr_pf_data_val'

POSTGRES_ADDRESS = 'localhost'
POSTGRES_USERNAME = 'postgres' 
POSTGRES_PASSWORD = 'jonp8UMs8qDV4jEcwOC0'
POSTGRES_DBNAME = 'thesis'

postgres_str = ('postgresql://{username}:{password}@{ipaddress}/{dbname}' 
                .format(username=POSTGRES_USERNAME,
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_ADDRESS,
                        dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str)

X_val_pf.to_sql(table, cnx, if_exists='replace')

with cnx.connect() as con:
    for col in ['t_taxi_end', 't_predict', 'dtype']:
        con.execute("CREATE INDEX ix_{}_{} ON {} ({})".format(table, col, table, col))

## Performance Feedback Tuning (lookback_min, factor)

In [12]:
best = {
    "value": no_fs_score,
    "lookback_min": 0,
    "factor": 0,
}

dtype = 'VALIDATE'

def avg_lookback_error(interval_m, table,  dtype):
    query = """
        SELECT a.index, AVG(b.error)
        FROM
            {table} AS a
            LEFT JOIN
            {table} AS b
            ON (a.t_predict > b.t_taxi_end) AND (a.t_predict - interval '{interval_m} minutes' < b.t_taxi_end)
        WHERE a.dtype='{dtype}'
        GROUP BY a.index;
    """.format(table=table, interval_m=interval_m, dtype=dtype)
    return pd.read_sql_query(query, cnx, index_col='index').fillna(0).values[:,0]

short_term_correction  = 0.08 * avg_lookback_error(55.28157368274357, table, dtype)
long_term_correction = 0.14 * avg_lookback_error(10147.315800050748, table, dtype)

for i, interval_m in enumerate(np.logspace(np.log10(30), np.log10(21*24*60), num=20)):
    print(i, interval_m)    

    new_correction = avg_lookback_error(interval_m, table, dtype)
    
    for factor in np.linspace(0,1,101):
#         y_val_pf_pred = (y_val_pred - factor * avg_td_error['avg']).to_numpy()

        y_val_pf_pred = (y_val_pred - short_term_correction - long_term_correction - factor * new_correction)

        report = model_eval(y_val, y_val_pf_pred, verbose=False)
        if report['% <2 min'] > best['value']:
            print("new optimium", interval_m, factor, report['% <2 min'])
            best = {
                "value": report['% <2 min'],
                "lookback_min": interval_m,
                "factor": factor
            }

model_eval(y_val, (y_val_pred - short_term_correction - long_term_correction))

{'RMSE': 184.7819630215103, 'MAE': 124.8550352013978, '% <2 min': 61.69298699232269, '% <5 min': 92.4181550578782, '% <7 min': 97.05238871872389, 'time': '2021-04-29 10:49:13'}


{'RMSE': 184.7819630215103,
 'MAE': 124.8550352013978,
 '% <2 min': 61.69298699232269,
 '% <5 min': 92.4181550578782,
 '% <7 min': 97.05238871872389,
 'time': '2021-04-29 10:49:13'}

# Table: Performance Feedback Validation Data

In [24]:
out = pd.DataFrame([
    {
        'name': 'no_pf_train', 
        'RMSE': 183.6191804345149, 
        'MAE': 124.38923499757205, 
        '% <2 min': 61.56967262023151, 
        '% <5 min': 92.66478380206054, 
        '% <7 min': 97.1677473248737, 
        'time': '2021-04-29 10:43:44'
    },
    {
        'RMSE': 184.7819630215103,
        'MAE': 124.8550352013978,
        '% <2 min': 61.69298699232269,
        '% <5 min': 92.4181550578782,
        '% <7 min': 97.05238871872389,
        'time': '2021-04-29 10:49:13'
    }  
])
print(out[["% <2 min", "MAE", "RMSE", "% <5 min", "% <7 min"]].round(2).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  \% <2 min &     MAE &    RMSE &  \% <5 min &  \% <7 min \\
\midrule
0 &     61.57 &  124.39 &  183.62 &     92.66 &     97.17 \\
1 &     61.69 &  124.86 &  184.78 &     92.42 &     97.05 \\
\bottomrule
\end{tabular}



# Performance Feedback Test Data

## No Performance Feedback

In [13]:
from sklearn.linear_model import HuberRegressor
from sklearn.pipeline import Pipeline

alpha = .4
epsilon = 1.

regr = HuberRegressor(
    max_iter=10000,
    alpha=alpha,
    epsilon=epsilon,
)

X_cols_sel = [
    'actype',
    'depgnr',
    'n_dep',
    'dew',
    'lightning',
    'rvr5000_2000',
    'plr1',
    'plr2',
    'local_mod',
    'trwy_ext',
]

t0 = time.time()
model = Pipeline(steps=[("preprocessor", gen_preprocessor(X_cols_sel)),
                       ("regressor", regr)])

model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

model_eval(y_test, y_test_pred)

{'RMSE': 172.42617267715704, 'MAE': 123.5536063902857, '% <2 min': 60.91055552240587, '% <5 min': 92.78198778765639, '% <7 min': 97.44813731925134, 'time': '2021-04-29 10:49:48'}


{'RMSE': 172.42617267715704,
 'MAE': 123.5536063902857,
 '% <2 min': 60.91055552240587,
 '% <5 min': 92.78198778765639,
 '% <7 min': 97.44813731925134,
 'time': '2021-04-29 10:49:48'}

## Calculate and Evaluate Performance Feedback

In [14]:
table = "lr_pf_data_test"
dtype = "TEST"

X_test_pf = pd.concat([X_train, X_val, X_test])[['t_taxi_end', 't_predict', 'dtype']]
X_test_pf['t_taxi_end'] = pd.to_datetime(X_test_pf['t_taxi_end'])
X_test_pf['t_predict'] = pd.to_datetime(X_test_pf['t_predict'])
X_test_pf['error'] = pd.concat([y_train, y_val, y_test]) - np.hstack([y_train_pred, y_val_pred, y_test_pred])

X_test_pf.to_sql(table, cnx, if_exists='replace')

with cnx.connect() as con:
    for col in ['t_taxi_end', 't_predict', 'dtype']:
        con.execute("CREATE INDEX ix_{}_{} ON {} ({})".format(table, col, table, col))

In [15]:
short_term_correction  = 0.08 * avg_lookback_error(55.28157368274357, table, dtype)
long_term_correction = 0.14 * avg_lookback_error(10147.315800050748, table, dtype)
y_test_pf_pred = (y_test_pred - short_term_correction - long_term_correction)

model_eval(y_test, y_test_pf_pred)

{'RMSE': 173.55815545922746, 'MAE': 123.99405039968248, '% <2 min': 60.926467370765955, '% <5 min': 92.60099051256041, '% <7 min': 97.33874336177576, 'time': '2021-04-29 10:51:30'}


{'RMSE': 173.55815545922746,
 'MAE': 123.99405039968248,
 '% <2 min': 60.926467370765955,
 '% <5 min': 92.60099051256041,
 '% <7 min': 97.33874336177576,
 'time': '2021-04-29 10:51:30'}

# Table: Performance Feedback Test Data

In [25]:
out = pd.DataFrame([
    {
        'RMSE': 172.42617267715704,
        'MAE': 123.5536063902857,
        '% <2 min': 60.91055552240587,
        '% <5 min': 92.78198778765639,
        '% <7 min': 97.44813731925134,
        'time': '2021-04-29 10:49:48'
    },
    {
        'RMSE': 173.55815545922746,
        'MAE': 123.99405039968248,
        '% <2 min': 60.926467370765955,
        '% <5 min': 92.60099051256041,
        '% <7 min': 97.33874336177576,
        'time': '2021-04-29 10:51:30'
    }  
])
print(out[["% <2 min", "MAE", "RMSE", "% <5 min", "% <7 min"]].round(2).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  \% <2 min &     MAE &    RMSE &  \% <5 min &  \% <7 min \\
\midrule
0 &     60.91 &  123.55 &  172.43 &     92.78 &     97.45 \\
1 &     60.93 &  123.99 &  173.56 &     92.60 &     97.34 \\
\bottomrule
\end{tabular}



# Validate (Algorithm not valdiation data)

In [16]:
from sklearn.linear_model import HuberRegressor
from sklearn.pipeline import Pipeline

alpha = .4
epsilon = 1.

regr = HuberRegressor(
    max_iter=10000,
    alpha=alpha,
    epsilon=epsilon,
)

X_cols_sel = [
    'actype',
    'depgnr',
    'n_dep',
    'dew',
    'lightning',
    'rvr5000_2000',
    'plr1',
    'plr2',
    'local_mod',
    'trwy_ext',
]

t0 = time.time()
model = Pipeline(steps=[("preprocessor", gen_preprocessor(X_cols_sel)),
                       ("regressor", regr)])

model.fit(pd.concat([X_train]), pd.concat([y_train]))

y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)

model_eval(y_val, y_val_pred)
# 55972791280481

{'RMSE': 183.6162753471294, 'MAE': 124.38756391693973, '% <2 min': 61.56967262023151, '% <5 min': 92.66478380206054, '% <7 min': 97.1677473248737, 'time': '2021-04-29 10:51:56'}


{'RMSE': 183.6162753471294,
 'MAE': 124.38756391693973,
 '% <2 min': 61.56967262023151,
 '% <5 min': 92.66478380206054,
 '% <7 min': 97.1677473248737,
 'time': '2021-04-29 10:51:56'}

In [17]:
table = "lr_pf_data_val"
dtype = "VALIDATE"

X_val_pf = pd.concat([X_train, X_val])[['t_taxi_end', 't_predict', 'dtype']]
X_val_pf['t_taxi_end'] = pd.to_datetime(X_val_pf['t_taxi_end'])
X_val_pf['t_predict'] = pd.to_datetime(X_val_pf['t_predict'])
X_val_pf['error'] = pd.concat([y_train, y_val]) - np.hstack([y_train_pred, y_val_pred])

X_val_pf.to_sql(table, cnx, if_exists='replace')

with cnx.connect() as con:
    for col in ['t_taxi_end', 't_predict', 'dtype']:
        con.execute("CREATE INDEX ix_{}_{} ON {} ({})".format(table, col, table, col))

In [18]:
short_term_correction  = 0.08 * avg_lookback_error(55.28157368274357, table, dtype)
long_term_correction = 0.14 * avg_lookback_error(10147.315800050748, table, dtype)
y_val_pf_pred = (y_val_pred - short_term_correction - long_term_correction)

model_eval(y_val, y_val_pf_pred)

{'RMSE': 184.77881790437058, 'MAE': 124.85329489674461, '% <2 min': 61.690998050837344, '% <5 min': 92.41616611639284, '% <7 min': 97.05437766020924, 'time': '2021-04-29 10:53:36'}


{'RMSE': 184.77881790437058,
 'MAE': 124.85329489674461,
 '% <2 min': 61.690998050837344,
 '% <5 min': 92.41616611639284,
 '% <7 min': 97.05437766020924,
 'time': '2021-04-29 10:53:36'}